<a href="https://colab.research.google.com/github/debasishsarangi88/TSAI_END2.0/blob/session4/END2_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [17]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 4201
Number of testing examples: 25000


In [16]:
print(vars(train_data.examples[0]))

{'text': ['At', 'first', 'i', 'did', "n't", 'think', 'that', 'Ben', 'Affleck', 'could', 'really', 'pull', 'off', 'a', 'funny', 'Christmas', 'movie', ',', ',', 'boy', 'was', 'i', 'wrong', ',', 'my', 'daughter', 'invited', 'me', 'to', 'watch', 'this', 'with', 'her', 'and', 'i', 'was', 'not', 'disappointed', 'at', 'all', '.', 'James', 'Gandolfini', 'was', 'funny', ',', ',', 'i', 'really', 'liked', 'Christina', 'Appelagate', ',', 'and', 'Catherine', 'O', "'", 'Hara', 'was', 'good', 'too', ',', 'the', 'storyline', 'is', 'what', 'really', 'sold', 'me', ',', ',', 'i', 'mean', ',', ',', 'too', 'put', 'up', 'with', 'family', ',', ',', 'at', 'the', 'table', 'for', 'people', 'you', 'only', 'hardly', 'see', 'but', 'once', 'or', 'twice', 'a', 'year', ',', ',', 'and', 'probably', 'do', "n't", 'get', 'along', 'with', 'anyway', ',', ',', 'you', 'really', 'do', 'need', 'as', 'much', 'alcohol', 'as', 'you', "'re", 'system', 'can', 'stand', 'to', 'deal', 'with', 'Christmas', ',', ',', 'so', 'i', 'thought

In [14]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [15]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 4201
Number of validation examples: 1801
Number of testing examples: 25000


In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 142205), (',', 136050), ('.', 116592), ('and', 77157), ('a', 76834), ('of', 71104), ('to', 65697), ('is', 53742), ('in', 43468), ('I', 38351), ('it', 37797), ('that', 34572), ('"', 31296), ("'s", 30572), ('this', 30023), ('-', 26062), ('/><br', 25178), ('was', 24695), ('as', 21327), ('movie', 21250)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
# import torch.nn as nn

# class RNN(nn.Module):
#     def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
#         super().__init__()
        
#         self.embedding = nn.Embedding(input_dim, embedding_dim)
        
#         self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
#         self.fc = nn.Linear(hidden_dim, output_dim)
        
#     def forward(self, text):

#         #text = [sent len, batch size]
        
#         embedded = self.embedding(text)
        
#         #embedded = [sent len, batch size, emb dim]
        
#         output, hidden = self.rnn(embedded)
        
#         #output = [sent len, batch size, hid dim]
#         #hidden = [1, batch size, hid dim]
        
#         assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
#         return self.fc(hidden.squeeze(0))



from torch import nn

class LSTM(nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, drop_prct = 0.5):
        super().__init__()
        # params: "n_" means dimension
        self.input_dim = input_dim     # number of unique words in vocabulary
        self.n_layers = n_layers   # number of LSTM layers 
        self.n_hidden = hidden_dim   # number of hidden nodes in LSTM
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first = True, dropout = drop_prct)
        self.dropout = nn.Dropout(drop_prct)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward (self, input_words):
                                             # INPUT   :  (batch_size, seq_length)
        embedded_words = self.embedding(input_words)    # (batch_size, seq_length, n_embed)
        lstm_out, h = self.lstm(embedded_words)         # (batch_size, seq_length, n_hidden)
        lstm_out = self.dropout(lstm_out)
        lstm_out = lstm_out.contiguous().view(-1, self.n_hidden) # (batch_size*seq_length, n_hidden)
        fc_out = self.fc(lstm_out)                      # (batch_size*seq_length, n_output)
        sigmoid_out = self.sigmoid(fc_out)              # (batch_size*seq_length, n_output)
        sigmoid_out = sigmoid_out.view(batch_size, -1)  # (batch_size, seq_length*n_output)
        
        # extract the output of ONLY the LAST output of the LAST element of the sequence
        sigmoid_last = sigmoid_out[:, -1]               # (batch_size, 1)
        
        return sigmoid_last, h
    
    
    def init_hidden (self, batch_size):  # initialize hidden weights (h,c) to 0
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        weights = next(self.parameters()).data
        h = (weights.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
             weights.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        
        return h




In [ ]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS=2

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,N_LAYERS)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 10m 45s
	Train Loss: 0.694 | Train Acc: 49.16%
	 Val. Loss: 0.696 |  Val. Acc: 50.36%
Epoch: 02 | Epoch Time: 10m 40s
	Train Loss: 0.693 | Train Acc: 50.31%
	 Val. Loss: 0.695 |  Val. Acc: 49.04%
Epoch: 03 | Epoch Time: 10m 39s
	Train Loss: 0.693 | Train Acc: 50.50%
	 Val. Loss: 0.696 |  Val. Acc: 50.02%
Epoch: 04 | Epoch Time: 10m 43s
	Train Loss: 0.693 | Train Acc: 50.63%
	 Val. Loss: 0.696 |  Val. Acc: 50.15%
Epoch: 05 | Epoch Time: 10m 38s
	Train Loss: 0.693 | Train Acc: 50.37%
	 Val. Loss: 0.695 |  Val. Acc: 49.17%


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.706 | Test Acc: 47.98%
